In [16]:
import glob
import os
import pandas as pd
import numpy as np
import json
import docx
from bs4 import BeautifulSoup as bs
import requests
from tqdm.auto import tqdm, trange
from pymystem3 import Mystem
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize
from nltk.stem import *
from nltk.corpus import stopwords
import string
import re
import pyLDAvis
import pyLDAvis.lda_model
import pymorphy2

In [18]:
import os
import json
import pandas as pd
from zipfile import ZipFile

In [23]:
# import os
# import glob
# import json
# import pandas as pd

# # это список json файлов
# list_json = glob.glob('C:/Users/Admin/Downloads/data/data/*.json')
# import pandas as pd
# data_list = []
# for file in list_json:   
#    with open(file, "r", encoding = 'utf-8') as f:
#        data=pd.read_json(f)
#        data_list.append(data)

# # конвертируем список DataFrame в один DataFrame
# df = pd.concat(data_list, ignore_index=True)
# df.head(5)

In [25]:
import os
import glob
import json
# это список json файлов
list_json = glob.glob('data/*.json')

In [26]:
import pandas as pd
data_list = []
for file in list_json:    
    with open(file, "r", encoding = 'utf-8') as f:
        data=pd.read_json(file)

In [27]:
data.head(5)

,id,text,title
0,44228062,Reinder Pieter (Reind) van de Riet (22 April 1...,Reinder van de Riet
1,44228065,The 2014–15 EHF Challenge Cup was the 18th edi...,2014–15 EHF Challenge Cup
2,44228107,"De Haag () is a post mill in Beuningen, Gelder...","De Haag, Beuningen"
3,44228138,Horst-Dieter Kolletschke (born 29 February 195...,Horst-Dieter Kolletschke
4,44228148,The first terrestrial television broadcast sig...,List of television stations in Nigeria


In [28]:
filtered_data = data[data['id'] == 7751062]

In [31]:
data.isnull().sum()

id       0
text     0
title    0
dtype: int64

In [32]:
def remove_tags(text):
    # удаление тегов разметки
    text = re.sub(r"<[^>]+>", "", str(text), flags=re.S)
    # удаление последовательностей \r и \n
    text = re.sub(r"\r|\n", " ", str(text), flags=re.S)
    # удаление последовательностей \b
    text = re.sub(r"\b", "", str(text), flags=re.S)
    text = re.sub(r" +", " ", str(text), flags=re.S)
    text = re.sub(r"^\d+$", "", str(text), flags=re.S)
    return text

def remove_punct(text):
    import string
    # удаляем пунктуацию и  
    # склеиваем символы, приводя слова к нижнему регистру
    return ''.join([c for c in text if c not in string.punctuation or c == '-']).lower()

def remove_multiple_spaces(text): #Функция удаления пробелов
    return re.sub(r'\s+', ' ', text, flags=re.I)

In [34]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [36]:
import string
from pymystem3 import Mystem
from tqdm.auto import tqdm, trange
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer as snow 

def remove_punctuation(text): #Функция удаления знаков препинания
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])
def remove_numbers(text): #Функция удаления чисел
    return ''.join([i if not i.isdigit() else ' ' for i in text])
def remove_multiple_spaces(text): #Функция удаления пробелов
    return re.sub(r'\s+', ' ', text, flags=re.I)

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian") #создание списка стоп-слов
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д', 'это']) #Дополнение стоп-слов

In [ ]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df['text'])]

In [ ]:
# для преобразования чисел в слова
import decimal
import re

units = (
    u'ноль',

    (u'один', u'одна'),
    (u'два', u'две'),

    u'три', u'четыре', u'пять',
    u'шесть', u'семь', u'восемь', u'девять'
)

teens = (
    u'десять', u'одиннадцать',
    u'двенадцать', u'тринадцать',
    u'четырнадцать', u'пятнадцать',
    u'шестнадцать', u'семнадцать',
    u'восемнадцать', u'девятнадцать'
)

tens = (
    teens,
    u'двадцать', u'тридцать',
    u'сорок', u'пятьдесят',
    u'шестьдесят', u'семьдесят',
    u'восемьдесят', u'девяносто'
)

hundreds = (
    u'сто', u'двести',
    u'триста', u'четыреста',
    u'пятьсот', u'шестьсот',
    u'семьсот', u'восемьсот',
    u'девятьсот'
)

orders = (
    ((u'тысяча', u'тысячи', u'тысяч'), 'f'),
    ((u'миллион', u'миллиона', u'миллионов'), 'm'),
    ((u'миллиард', u'миллиарда', u'миллиардов'), 'm')
)

minus = u'минус'


def thousand(rest, sex):
    """Converts numbers from 19 to 999"""
    prev = 0
    plural = 2
    name = []
    use_teens = rest % 100 >= 10 and rest % 100 <= 19
    if not use_teens:
        data = ((units, 10), (tens, 100), (hundreds, 1000))
    else:
        data = ((teens, 10), (hundreds, 1000))
    for names, x in data:
        cur = int(((rest - prev) % x) * 10 / x)
        prev = rest % x
        if x == 10 and use_teens:
            plural = 2
            name.append(teens[cur])
        elif cur == 0:
            continue
        elif x == 10:
            name_ = names[cur]
            if isinstance(name_, tuple):
                name_ = name_[0 if sex == 'm' else 1]
            name.append(name_)
            if cur >= 2 and cur <= 4:
                plural = 1
            elif cur == 1:
                plural = 0
            else:
                plural = 2
        else:
            name.append(names[cur-1])
    return plural, name


def num2text(num, main_units=((u'', u'', u''), 'm')):
    """
    http://ru.wikipedia.org/wiki/Gettext#.D0.9C.D0.BD.D0.BE.D0.B6.D0.B5.D1.81.\
    D1.82.D0.B2.D0.B5.D0.BD.D0.BD.D1.8B.D0.B5_.D1.87.D0.B8.D1.81.D0.BB.D0.B0_2
    """
    _orders = (main_units,) + orders
    if num == 0:
        return ' '.join((units[0], _orders[0][0][2])).strip() # ноль

    rest = abs(num)
    ord = 0
    name = []
    while rest > 0:
        plural, nme = thousand(rest % 1000, _orders[ord][1])
        if nme or ord == 0:
            name.append(_orders[ord][0][plural])
        name += nme
        rest = int(rest / 1000)
        ord += 1
    if num < 0:
        name.append(minus)
    name.reverse()
    return ' '.join(name).strip()


def decimal2text(value, places=2,
                 int_units=(('', '', ''), 'm'),
                 exp_units=(('', '', ''), 'm')):
    value = decimal.Decimal(value)
    q = decimal.Decimal(10) ** -places

    integral, exp = str(value.quantize(q)).split('.')
    return u'{} {}'.format(
        num2text(int(integral), int_units),
        num2text(int(exp), exp_units))



def getnumbers(s):
    nums = re.findall(r'\d*\.\d+|\d+', s)
    nums = [float(i) for i in nums]
    for x in nums:
        s=s.replace(str(x), decimal2text(float(x)))
    nums=[]
    nums = re.findall(r'\d+', s)
    nums = [int(i) for i in nums]
    for x in nums:
        s=s.replace(str(x), num2text(int(x)))
    return s

In [ ]:
# для удаления пунктуации
def remove_punct(text):
    import string
    # удаляем пунктуацию и  
    # склеиваем символы, приводя слова к нижнему регистру
    return ''.join([c for c in text if c not in string.punctuation or c == '-']).lower()

In [ ]:
# для лематизации
@lru_cache(maxsize=None)

In [ ]:
import pymorphy2
from nltk.corpus import stopwords
from functools import lru_cache

# кешируем слова для быстрого доступа к ним в других случаях
@lru_cache(maxsize=None) 
def norm_form(word):
    return morph.parse(word)[0].normal_form

new_nomenclature = list()
morph = pymorphy2.MorphAnalyzer()
for text in tqdm(data_all['Номенклатура']):
    # приведём все слова к нормальной форме и удалим малозначимые слова
    new_nomenclature.append(' '.join([norm_form(word) for word in text.split(' ') \
                 if word not in stopwords.words('russian') or word == 'то']).strip())
data_all['НоменклатураНовая'] = new_nomenclature

In [ ]:
# для html-тегов
def remove_tags(text):
    # удаление тегов разметки
    text = re.sub(r"<[^>]+>", "", str(text), flags=re.S)
    # удаление последовательностей \r и \n
    text = re.sub(r"\r|\n", " ", str(text), flags=re.S)
    # удаление последовательностей \b
    text = re.sub(r"\b", "", str(text), flags=re.S)
    text = re.sub(r" +", " ", str(text), flags=re.S)
    text = re.sub(r"^\d+$", "", str(text), flags=re.S)
    return text